In [1]:
# imports and installations
!nvidia-smi
!pip install -U torch
!pip install transformers
!pip install tensorboardX
!pip install selenium
!pip install selenium-stealth
!pip install beautifulsoup4
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!sudo apt install tesseract-ocr
!pip install pytesseract
!wget https://norvig.com/big.txt

import io, re, requests, sys, time, torch
from collections import Counter
import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')

import pytesseract
from PIL import Image

import requests #needed to load the page for BS4
from bs4 import BeautifulSoup
import pandas as pd #Using panda to create our dataframe

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import pipeline
from transformers import (
    AlbertConfig,
    AlbertForQuestionAnswering,
    AlbertTokenizer,
    squad_convert_examples_to_features
)
from transformers.data.processors.squad import SquadResult, SquadV2Processor, SquadExample
from transformers.data.metrics.squad_metrics import compute_predictions_logits

sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
import selenium
from selenium import webdriver #webdriver is our tool to interact with the webpage
from selenium_stealth import stealth

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.

  Using cached urllib3-1.25.11-py2.py3-none-any.whl (127 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.9
    Uninstalling urllib3-1.26.9:
      Successfully uninstalled urllib3-1.26.9
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
selenium 4.1.5 requires urllib3[secure,socks]~=1.26, but you have urllib3 1.25.11 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
  Using cached urllib3-1.26.9-py2.py3-none-any.whl (138 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
ERROR: pip's dependency resolver does not

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# **Data Retrieval**

In [2]:
# return a driver with stealth mode
def get_driver():
  options = webdriver.ChromeOptions()
  options.add_argument("start-maximized")
  options.add_argument("--headless")
  options.add_argument("--incognito")
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-extensions')
  options.add_experimental_option("excludeSwitches", ["enable-automation"])
  options.add_experimental_option('useAutomationExtension', False)
  driver = webdriver.Chrome('chromedriver', options=options)

  stealth(driver,
          languages=["en-US", "en"],
          vendor="Google Inc.",
          platform="Win32",
          webgl_vendor="Intel Inc.",
          renderer="Intel Iris OpenGL Engine",
          fix_hairline=True,
          )
  
  return driver

In [3]:
# extracting urls from webpage
driver = get_driver()
url = "https://ddr.densho.org/collections/#ddr-sbbt"
pattern = re.compile("https://ddr.densho.org/ddr-.*")

driver.get(url)

urls = list()
# identify elements with tagname <a> and has href attribute
elems = driver.find_elements_by_xpath("//a[@href]")
# traverse list
for elem in elems: 
  # fetch href attribute
  url = elem.get_attribute("href")
  # removing duplicates and adding to list and filtering urls
  if url not in urls and bool(pattern.match(url)):
    urls.append(url)
print(urls)

driver.quit()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  # Remove the CWD from sys.path while we load stuff.


['https://ddr.densho.org/ddr-ajah-1/', 'https://ddr.densho.org/ddr-chi-1/', 'https://ddr.densho.org/ddr-csujad-1/', 'https://ddr.densho.org/ddr-csujad-2/', 'https://ddr.densho.org/ddr-csujad-3/', 'https://ddr.densho.org/ddr-csujad-4/', 'https://ddr.densho.org/ddr-csujad-5/', 'https://ddr.densho.org/ddr-csujad-6/', 'https://ddr.densho.org/ddr-csujad-7/', 'https://ddr.densho.org/ddr-csujad-8/', 'https://ddr.densho.org/ddr-csujad-9/', 'https://ddr.densho.org/ddr-csujad-10/', 'https://ddr.densho.org/ddr-csujad-11/', 'https://ddr.densho.org/ddr-csujad-12/', 'https://ddr.densho.org/ddr-csujad-13/', 'https://ddr.densho.org/ddr-csujad-14/', 'https://ddr.densho.org/ddr-csujad-15/', 'https://ddr.densho.org/ddr-csujad-16/', 'https://ddr.densho.org/ddr-csujad-17/', 'https://ddr.densho.org/ddr-csujad-18/', 'https://ddr.densho.org/ddr-csujad-19/', 'https://ddr.densho.org/ddr-csujad-20/', 'https://ddr.densho.org/ddr-csujad-21/', 'https://ddr.densho.org/ddr-csujad-22/', 'https://ddr.densho.org/ddr-csu

In [31]:
# utility functions

# function to extract text from image
def img_to_text(url, headers=None):
  headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
  r = requests.get(url, headers=headers)
  img = Image.open(io.BytesIO(r.content))
  text = pytesseract.image_to_string(img)
  text = clean_text(text, auto_correction=True)
  return text

# function to extract text from html 
def html_to_text(html, blacklist=None):
  soup = BeautifulSoup(html, 'lxml')
  text = soup.find_all(text=True)

  # Remove unwanted tag elements:
  cleaned_text = ''
  blacklist = [
      '[document]', 
      'noscript',
      'header',
      'html',
      'meta',
      'head', 
      'input',
      'script',
      'style',]

  # Then we will loop over every item in the extract text and make sure that the beautifulsoup4 tag
  # is NOT in the blacklist
  for item in text:
      if item.parent.name not in blacklist:
          cleaned_text += '{} '.format(item)
          
  # Remove any tab separation and strip the text:
  cleaned_text = clean_text(cleaned_text)
  return cleaned_text

# spell correction code
def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('big.txt').read()))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def get_corrected_sentence(sentence):
   return ' '.join([correction(word) for word in sentence.split()])

# data cleaning
def clean_text(text, auto_correction=False):
  text = text.replace('\t', '')
  text = text.strip()
  text = text.lower()
  if auto_correction:
    text = get_corrected_sentence(text)
  return text


In [27]:
# extracting data from page
def get_content(html):
  soup = BeautifulSoup(html, 'lxml')
  right_col = soup.select('div#right-col')
  left_col = soup.select('div#left-col')

  description = html_to_text(''.join([tag.text for tag in right_col]))
  # description = re.findall(r"\nDESCRIPTION\n.*\n", description)[0] if description else ''

  img_url = None
  for tag in left_col:
    if tag.find_all('img'):
      img_url = tag.find('img').get('src')
    if tag.find_all('a'):
      img_url = tag.find('a').get('href')
  img_text = img_to_text(img_url) if img_url else ''
  
  return f"{img_text} {description}"

In [32]:
# extracting text from html for all the urls collected from the webpage
contents = list()
for url in urls:
  try:
    driver = get_driver()
    driver.get(url)
    contents.append(get_content(driver.page_source))
    driver.quit()
  except:
    contents.append('')
print(contents)

[' addthis sharing buttonsshare to facebookfacebookshare to twittertwittershare to moreaddthis\n\n\npartner\n\n\n\n\n\n\n\nalameda japanese american history project visit partner\n\n\n\n\ncollection id\nddr-ajah-1\ndescription\nthis collection consists of interviews with nisei and sansei talking about the japanese american community in alameda, california. the project partners include the internet archive (archive.org), the alameda library, the buddhist temple of alameda, buena vista united methodist church, rhythmix cultural works, and densho. the alameda museum also contributed but was not a partner in the process. the goal of the alameda japanese american history project is to reach and educate the world wide general public.\n\n\n\ninclusive unit date\n2021-2022\n\n\nbulk unit date\n2021-2022\n\n\n\nphysical description\n\ncontributor\nalameda japanese american history project\npreferred citation\ncourtesy of the alameda japanese american history project\nrights\n\n\nthis work is li

In [37]:
df = pd.DataFrame({
  'url': urls, 
  'content': contents
})

df.to_csv('ddr_crude.csv', index=False)

# **Text Summarization**

In [10]:
df = pd.read_csv('ddr_crude.csv')

summarizer = pipeline("summarization")

def get_summary(content):
  try: 
    summary = summarizer(content)
  except:
    summary = summarizer(content[:1023])
  return summary[0]['summary_text']

df['content'] = df.content.apply(get_summary)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)
Token indices sequence length is longer than the specified maximum sequence length for this model (1268 > 1024). Running this sequence through the model will result in indexing errors
Your max_length is set to 142, but you input_length is only 133. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=66)
Your max_length is set to 142, but you input_length is only 132. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=66)
Your max_length is set to 142, but you input_length is only 141. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=70)
Your max_length is set to 142, but you input_length is only 130. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=65)
Your max_length is set to 142, but you input_length is only 140. You might cons

In [11]:
df.to_csv('ddr_summarized.csv', index=False)

# **Question Answering**

In [16]:
# READER NOTE: Set this flag to use own model, or use pretrained model in the Hugging Face repository
use_own_model = False

if use_own_model:
  model_name_or_path = "/content/model_output"
else:
  model_name_or_path = "ktrapeznikov/albert-xlarge-v2-squad-v2"

output_dir = ""

# Config
n_best_size = 1
max_answer_length = 30
do_lower_case = True
null_score_diff_threshold = 0.0

def to_list(tensor):
    return tensor.detach().cpu().tolist()

# Setup model
config_class, model_class, tokenizer_class = (
    AlbertConfig, AlbertForQuestionAnswering, AlbertTokenizer)
config = config_class.from_pretrained(model_name_or_path)
tokenizer = tokenizer_class.from_pretrained(
    model_name_or_path, do_lower_case=True)
model = model_class.from_pretrained(model_name_or_path, config=config)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

processor = SquadV2Processor()

def run_prediction(question_texts, context_text):
    """Setup function to compute predictions"""
    examples = []

    for i, question_text in enumerate(question_texts):
        example = SquadExample(
            qas_id=str(i),
            question_text=question_text,
            context_text=context_text,
            answer_text=None,
            start_position_character=None,
            title="Predict",
            is_impossible=False,
            answers=None,
        )

        examples.append(example)

    features, dataset = squad_convert_examples_to_features(
        examples=examples,
        tokenizer=tokenizer,
        max_seq_length=384,
        doc_stride=128,
        max_query_length=64,
        is_training=False,
        return_dataset="pt",
        threads=1,
    )

    eval_sampler = SequentialSampler(dataset)
    eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=10)

    all_results = []

    for batch in eval_dataloader:
        model.eval()
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2],
            }

            example_indices = batch[3]

            outputs = model(**inputs)

            for i, example_index in enumerate(example_indices):
                eval_feature = features[example_index.item()]
                unique_id = int(eval_feature.unique_id)

                output = [to_list(output[i]) for output in outputs]

                start_logits, end_logits = output
                result = SquadResult(unique_id, start_logits, end_logits)
                all_results.append(result)

    output_prediction_file = "predictions.json"
    output_nbest_file = "nbest_predictions.json"
    output_null_log_odds_file = "null_predictions.json"

    predictions = compute_predictions_logits(
        examples,
        features,
        all_results,
        n_best_size,
        max_answer_length,
        do_lower_case,
        output_prediction_file,
        output_nbest_file,
        output_null_log_odds_file,
        False,  # verbose_logging
        True,  # version_2_with_negative
        null_score_diff_threshold,
        tokenizer,
    )

    return predictions

In [ ]:
context = " ".join(df.content)
questions = ["question 1", 
             "question 2",
             "question 3", 
             "question 4",
             "question 5", 
             "question 6"]

# Run method
predictions = run_prediction(questions, context)

# Print results
for key in predictions.keys():
  print(predictions[key])